In [1]:
import pandas as pd
import numpy as np
import math

from keras.layers import Embedding, Dense, LSTM, Dense, Input, concatenate
from keras.models import Model
from keras.utils import plot_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:
bot_accounts = pd.concat([pd.read_csv('social_spambots_1.csv'), pd.read_csv('social_spambots_2.csv'), pd.read_csv('social_spambots_3.csv')]).reset_index(drop=True)
clean_accounts = pd.read_csv('geniune_accounts.csv')

requiredColumns = ['listed_count', 'geo_enabled', 'verified', 'statuses_count', 'friends_count','followers_count', 'favourites_count', 'profile_use_background_image', 'protected', 'default_profile']
bot_accounts = bot_accounts[requiredColumns]
clean_accounts = clean_accounts[requiredColumns]
def clean_df(df):
    
    
    df['listed_count'] = df['listed_count'].apply(lambda x: 1 if x==x else 0)
    df['geo_enabled'] = df['geo_enabled'].apply(lambda x: 1 if x==x else 0)
    df['statuses_count'] = df['statuses_count'].apply(lambda x: 1 if x==x else 0)
    df['friends_count'] = df['friends_count'].apply(lambda x: 1 if x==x else 0)
    df['followers_count'] = df['followers_count'].apply(lambda x: 1 if x==x else 0)
    df['favourites_count'] = df['favourites_count'].apply(lambda x: 1 if x==x else 0)
    df['profile_use_background_image'] = df['profile_use_background_image'].apply(lambda x: 1 if x==x else 0)
    df['verified']=df['verified'].apply(lambda x: 1 if x==x else 0)
    df['is_protected']=df['protected'].apply(lambda x: 1 if x==x else 0)
    df['default_profile'] = df['default_profile'].apply(lambda x: 0 if x==x else 1)
    #df = df.rename(index=str, columns={"screen_name": "username", "statuses_count": "total_tweets", "friends_count": "total_following", "followers_count": "total_followers", "favourites_count": "total_likes"})
    return df[['listed_count', 'geo_enabled', 'verified', 'statuses_count', 'friends_count','followers_count', 'favourites_count', 'profile_use_background_image', 'is_protected', 'default_profile']]

bot_accounts = clean_df(bot_accounts)
clean_accounts = clean_df(clean_accounts)
#print(bot_accounts)

    



C:\Users\STUDENT\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [4]:
bot_accounts['BotOrNot'] = 1
clean_accounts['BotOrNot'] = 0
#print(bot_accounts)
#print(clean_accounts)

combined_df = pd.concat([bot_accounts, clean_accounts])

new_df = combined_df.sample(frac=1).reset_index(drop=True)

In [5]:
training_df = new_df[:int(combined_df.shape[0] * 0.8)]
test_df = new_df[int(combined_df.shape[0] * 0.8):]



columns_to_standardize = ['listed_count', 'geo_enabled', 'verified', 'statuses_count', 'friends_count','followers_count', 'favourites_count', 'profile_use_background_image', 'is_protected', 'default_profile']

training_df_mean = training_df[columns_to_standardize].mean()
training_df_std = training_df[columns_to_standardize].std()

#training_df[columns_to_standardize] = (training_df[columns_to_standardize] - training_df_mean)/training_df_std
#test_df[columns_to_standardize] = (test_df[columns_to_standardize] - training_df_mean)/training_df_std

# training_df_mean = training_df.mean()
# training_df_std = training_df.std()

# training_df = (training_df - training_df_mean)/training_df_std
# test_df = (test_df - training_df_mean)/training_df_std

# max_vals = training_df.max()

# training_df = training_df/max_vals
# test_df = test_df/max_vals

In [6]:
#X_train = training_df.drop(['BotOrNot', 'is_protected'], axis=1).values
#y_train = training_df['BotOrNot'].values.reshape(-1,1)
#print(X_train)
X_train = training_df
y_train = training_df['BotOrNot'].values.reshape(-1,1)

X_test = test_df
y_test = test_df['BotOrNot'].values.reshape(-1,1)
print()
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
sm = SMOTEENN()
X_train, y_train = sm.fit_sample(X_train, y_train)
#print(X_train)
from sklearn.ensemble import AdaBoostClassifier

# Import Support Vector Classifier
from sklearn.svm import SVC
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
svc=SVC(probability=True, kernel='linear')

# Create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=50, base_estimator=svc,learning_rate=1)

# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

C:\Users\STUDENT\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 1.0


In [7]:
inp = Input(shape=[11])

another = Dense(500, activation='relu')(inp)
another = Dense(200, activation='relu')(another)
another = Dense(1, activation='sigmoid')(another)

mod = Model(inp, another)
mod.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
training = mod.fit(x=X_train, y=y_train, batch_size=200, epochs=200, validation_data=(X_test, y_test))

Train on 7792 samples, validate on 1678 samples
Epoch 1/200
7792/7792 [==============================] - 0s 41us/step - loss: 0.2196 - acc: 0.9695 - val_loss: 0.0136 - val_acc: 0.9982
Epoch 2/200
7792/7792 [==============================] - 0s 15us/step - loss: 0.0040 - acc: 0.9999 - val_loss: 0.0013 - val_acc: 1.0000
Epoch 3/200
7792/7792 [==============================] - 0s 15us/step - loss: 7.7037e-04 - acc: 1.0000 - val_loss: 5.1883e-04 - val_acc: 1.0000
Epoch 4/200
7792/7792 [==============================] - 0s 16us/step - loss: 3.3975e-04 - acc: 1.0000 - val_loss: 2.5860e-04 - val_acc: 1.0000
Epoch 5/200
7792/7792 [==============================] - 0s 15us/step - loss: 1.6839e-04 - acc: 1.0000 - val_loss: 1.2627e-04 - val_acc: 1.0000
Epoch 6/200
7792/7792 [==============================] - 0s 15us/step - loss: 8.6068e-05 - acc: 1.0000 - val_loss: 6.8001e-05 - val_acc: 1.0000
Epoch 7/200
7792/7792 [==============================] - 0s 24us/step - loss: 5.0034e-05 - acc: 1.0000 -

7792/7792 [==============================] - 0s 16us/step - loss: 2.9438e-07 - acc: 1.0000 - val_loss: 3.1593e-07 - val_acc: 1.0000
Epoch 58/200
7792/7792 [==============================] - 0s 15us/step - loss: 2.8699e-07 - acc: 1.0000 - val_loss: 3.0825e-07 - val_acc: 1.0000
Epoch 59/200
7792/7792 [==============================] - 0s 19us/step - loss: 2.8054e-07 - acc: 1.0000 - val_loss: 3.0068e-07 - val_acc: 1.0000
Epoch 60/200
7792/7792 [==============================] - 0s 16us/step - loss: 2.7433e-07 - acc: 1.0000 - val_loss: 2.9383e-07 - val_acc: 1.0000
Epoch 61/200
7792/7792 [==============================] - 0s 16us/step - loss: 2.6873e-07 - acc: 1.0000 - val_loss: 2.8269e-07 - val_acc: 1.0000
Epoch 62/200
7792/7792 [==============================] - 0s 17us/step - loss: 2.5891e-07 - acc: 1.0000 - val_loss: 2.7619e-07 - val_acc: 1.0000
Epoch 63/200
7792/7792 [==============================] - 0s 18us/step - loss: 2.5402e-07 - acc: 1.0000 - val_loss: 2.7023e-07 - val_acc: 1.000

7792/7792 [==============================] - 0s 15us/step - loss: 1.2804e-07 - acc: 1.0000 - val_loss: 1.4109e-07 - val_acc: 1.0000
Epoch 114/200
7792/7792 [==============================] - 0s 17us/step - loss: 1.2734e-07 - acc: 1.0000 - val_loss: 1.4053e-07 - val_acc: 1.0000
Epoch 115/200
7792/7792 [==============================] - 0s 14us/step - loss: 1.2660e-07 - acc: 1.0000 - val_loss: 1.3972e-07 - val_acc: 1.0000
Epoch 116/200
7792/7792 [==============================] - 0s 14us/step - loss: 1.2610e-07 - acc: 1.0000 - val_loss: 1.3899e-07 - val_acc: 1.0000
Epoch 117/200
7792/7792 [==============================] - 0s 13us/step - loss: 1.2539e-07 - acc: 1.0000 - val_loss: 1.3805e-07 - val_acc: 1.0000
Epoch 118/200
7792/7792 [==============================] - 0s 15us/step - loss: 1.2458e-07 - acc: 1.0000 - val_loss: 1.3721e-07 - val_acc: 1.0000
Epoch 119/200
7792/7792 [==============================] - 0s 15us/step - loss: 1.2403e-07 - acc: 1.0000 - val_loss: 1.3663e-07 - val_acc:

7792/7792 [==============================] - 0s 14us/step - loss: 1.1154e-07 - acc: 1.0000 - val_loss: 1.1687e-07 - val_acc: 1.0000
Epoch 170/200
7792/7792 [==============================] - 0s 15us/step - loss: 1.1154e-07 - acc: 1.0000 - val_loss: 1.1687e-07 - val_acc: 1.0000
Epoch 171/200
7792/7792 [==============================] - 0s 14us/step - loss: 1.1154e-07 - acc: 1.0000 - val_loss: 1.1687e-07 - val_acc: 1.0000
Epoch 172/200
7792/7792 [==============================] - 0s 15us/step - loss: 1.1153e-07 - acc: 1.0000 - val_loss: 1.1665e-07 - val_acc: 1.0000
Epoch 173/200
7792/7792 [==============================] - 0s 16us/step - loss: 1.1148e-07 - acc: 1.0000 - val_loss: 1.1665e-07 - val_acc: 1.0000
Epoch 174/200
7792/7792 [==============================] - 0s 15us/step - loss: 1.1148e-07 - acc: 1.0000 - val_loss: 1.1651e-07 - val_acc: 1.0000
Epoch 175/200
7792/7792 [==============================] - 0s 16us/step - loss: 1.1148e-07 - acc: 1.0000 - val_loss: 1.1651e-07 - val_acc:

In [67]:
mod.save('my_model.h5')